In [1]:
import json

import numpy as np
import geopandas as gpd

In [2]:
import shapely.geometry

#### Load the data

In [3]:
data = json.load(open('../Data/mapaexekuci.cz/okresy.geojson', encoding='utf-8-sig'))
data = data['features']

In [4]:
pre_gdf = []
for entry in data:
    name = entry.get('properties').get('r')
    geometry = shapely.geometry.asShape(entry.get('geometry'))
    
    pre_gdf.append([name,geometry])
    

In [5]:
gdf = gpd.GeoDataFrame(pre_gdf)
gdf.columns = ['name','geometry']
gdf.crs = {'init': 'epsg:4326'}


#### Visualise

In [6]:
import matplotlib.pyplot as plt

In [7]:
# Krovak looks better
gdf = gdf.to_crs(epsg='5514')

In [10]:
gdf['shouldplot'] = False

In [55]:
for i,row in gdf.iterrows():
    
    gdf.loc[i,'shouldplot'] = True
    
    fig, ax = plt.subplots(1, figsize=(10, 6))
    ax.axis('off')
    gdf.plot(column = 'shouldplot', cmap='Blues', ax=ax, linewidth=0.8, edgecolor='#DFDFDF')
    
    plt.tight_layout()
    plt.savefig( r'images/' + str(i) + '.png')
    plt.close(fig)
    
    gdf.loc[i,'shouldplot'] = False

#### Generate Anki cards

In [64]:
import genanki

In [65]:
my_deck = genanki.Deck(
  1,            # Unique ID
  'Okresy CR')  # Deck name

In [66]:
my_model = genanki.Model(
  11,
  'Map-Answer Model',
  fields=[
    {'name': 'Question'},
    {'name': 'Answer'},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Question}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
  ])

In [67]:
for i in range(len(gdf)):
    my_note = genanki.Note(
      model = my_model,
      fields = ['<center><img src="images/' + str(i) + '.png"></center>', 
                '<center><font size = "+3">' + gdf.loc[i,'name'] + '</font></center>'])

    my_deck.add_note(my_note)

In [68]:
files = ['images/' + str(x) + '.png' for x in list(range(len(gdf)))]

In [69]:
my_package = genanki.Package(my_deck)
my_package.media_files = files

In [70]:
my_package.write_to_file('Okresy.apkg')